In [2]:
import spacepy.pycdf as pycdf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
L1_AUX = 'swis_2025Oct02T114501986/AL1_ASW91_L1_AUX_20250930_UNP_9999_999999_V01.cdf'
L1_TH1 = "swis_2025Oct02T114501986/AL1_ASW91_L1_TH1_20250930_UNP_9999_999999_V01.cdf"
L1_TH2 = 'swis_2025Oct02T114501986/AL1_ASW91_L1_TH2_20250930_UNP_9999_999999_V01.cdf'
L2_BLK = 'swis_2025Oct02T114501986/AL1_ASW91_L2_BLK_20250930_UNP_9999_999999_V02.cdf'
L2_TH1 = 'swis_2025Oct02T114501986/AL1_ASW91_L2_TH1_20250930_UNP_9999_999999_V02.cdf'
L2_TH2 = 'swis_2025Oct02T114501986/AL1_ASW91_L2_TH2_20250930_UNP_9999_999999_V02.cdf'

In [2]:
import cdflib
import spacepy.pycdf as pycdf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# File paths (update these to your actual file paths)
files = {
    'L1_AUX': L1_AUX,
    'L1_TH1': L1_TH1,
    'L1_TH2': L1_TH2,
    'L2_BLK': L2_BLK,
    'L2_TH1': L2_TH1,
    'L2_TH2': L2_TH2
}

In [ ]:
# pycdf.CDF(L2_BLK)
# give me into a csv file
from spacepy import pycdf
import pandas as pd

# Open the CDF file
cdf_file = pycdf.CDF(L2_BLK)

# Convert all variables into a dictionary
data = {var: cdf_file[var][...] for var in cdf_file}

# Convert dictionary into DataFrame
df = pd.DataFrame(data)

# Save as CSV
df.head(100).to_csv("L2_BLK.csv", index=False)

print("CSV file saved as L2_BLK.csv")



ValueError: Per-column arrays must each be 1-dimensional

In [10]:
from spacepy import pycdf
import numpy as np
import pandas as pd

def cdf_to_csv(cdf_path, csv_path, max_rows=None, pad_value=np.nan):
    """
    Convert a CDF to CSV handling multi-dimensional variables.
    - If a variable is 2D with shape (N, M) and N equals the chosen primary length, it expands to M columns.
    - If shapes don't match the primary length, shorter arrays are padded, longer arrays truncated.
    - mask arrays are filled with NaN.
    - max_rows: if set, will truncate the final DataFrame to that many rows.
    """
    cdf = pycdf.CDF(cdf_path)
    
    # read all variables first
    raw_vars = {}
    for var in cdf:
        arr = cdf[var][...]
        # convert masked arrays to normal arrays with NaN
        if hasattr(arr, 'filled'):
            arr = arr.filled(np.nan)
        arr = np.asarray(arr)
        raw_vars[var] = arr

    # determine a "primary length" (most likely the time dimension)
    lengths = []
    for arr in raw_vars.values():
        if arr.ndim == 0:
            lengths.append(1)
        else:
            # take first axis length
            lengths.append(arr.shape[0])
    primary_len = int(max(lengths)) if lengths else 0
    if primary_len == 0:
        raise ValueError("No data found in CDF.")

    # build columns
    columns = {}
    for name, arr in raw_vars.items():
        if arr.ndim == 0:
            # scalar: repeat to primary_len
            columns[name] = np.full(primary_len, arr.item(), dtype=object)
        elif arr.ndim == 1:
            # 1-D: pad or truncate to primary_len
            col = np.full(primary_len, pad_value, dtype=object)
            length = min(primary_len, arr.shape[0])
            col[:length] = arr[:length]
            columns[name] = col
        else:
            # multi-d array: try to align first axis with primary_len
            # if arr.shape[0] == primary_len, expand columns along axis 1
            if arr.shape[0] == primary_len:
                # expand second axis into separate columns
                # if more than 2 dims, flatten trailing dims into single dimension
                rest_shape = arr.shape[1:]
                ncols = int(np.prod(rest_shape))
                reshaped = arr.reshape((primary_len, ncols))
                for i in range(ncols):
                    col_name = f"{name}_{i}"
                    col = np.full(primary_len, pad_value, dtype=object)
                    col[:] = reshaped[:, i]
                    columns[col_name] = col
            elif arr.shape[-1] == primary_len:
                # maybe the time axis is the last axis; transpose then expand
                reshaped = np.moveaxis(arr, -1, 0)  # bring time axis to front
                ncols = int(np.prod(reshaped.shape[1:]))
                reshaped = reshaped.reshape((primary_len, ncols))
                for i in range(ncols):
                    col_name = f"{name}_{i}"
                    col = np.full(primary_len, pad_value, dtype=object)
                    col[:] = reshaped[:, i]
                    columns[col_name] = col
            else:
                # fallback: flatten whole array for each top index if possible
                # We'll create one column per row if first axis is >1 and <= primary_len,
                # else serialize each row to a string (safer).
                if arr.shape[0] <= primary_len:
                    # for row i, convert row to tuple/list and store
                    col = np.full(primary_len, pad_value, dtype=object)
                    for i in range(arr.shape[0]):
                        col[i] = arr[i].tolist()
                    columns[name] = col
                else:
                    # too many rows: truncate and serialize rows to lists
                    col = np.full(primary_len, pad_value, dtype=object)
                    for i in range(primary_len):
                        col[i] = arr[i].tolist()
                    columns[name] = col

    # build DataFrame
    df = pd.DataFrame(columns)

    # optional truncate to max_rows
    if max_rows is not None:
        df = df.iloc[:max_rows]

    # try to convert numpy datetime64 to pandas datetime if present
    for col in df.columns:
        if np.issubdtype(df[col].dtype, np.datetime64):
            df[col] = pd.to_datetime(df[col])

    # save to csv
    df.head(100).to_csv(csv_path, index=False)
    print(f"Saved CSV to: {csv_path} (rows: {len(df)})")

# Example usage:
# cdf_to_csv("L1_AUX.cdf", "L1_AUX.csv")
# If your file is already opened as a pycdf.CDF object (e.g., variable named L1_AUX),
# use L1_AUX.filename or pass the path string directly.

if __name__ == "__main__":
    # Replace with your file path
    cdf_path = L1_AUX
    csv_path = "L1_AUX.csv"
    cdf_to_csv(cdf_path, csv_path, max_rows=None)


Saved CSV to: L1_AUX.csv (rows: 172787)


In [3]:
def load_cdf(file_path):
    try:
        with cdflib.CDF(file_path) as cdf:
            variables = cdf.var_names()
            data = {}
            for var in variables:
                data[var] = cdf.varget(var)
                fillval = cdf.varattsget(var).get('FILLVAL', -1.0e31)
                if np.isscalar(data[var]):
                    continue
                data[var][data[var] == fillval] = np.nan
                if data[var].ndim > 1 and var not in ['sun_angle_tha1', 'sun_angle_tha2', 'angle_tha1', 'angle_tha2']:
                    data[var] = np.nanmean(data[var], axis=tuple(range(1, data[var].ndim)))
            return data
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

In [4]:
from src.processing_data import *

In [13]:
def calculate_cme_score(proton_density, proton_bulk_speed, proton_thermal, alpha_density):
    FILL_VALUE = -1.0e31
    k_B, m_p = 1.381e-23, 1.673e-27
    
    # Validity checks
    valid_proton = (proton_density > FILL_VALUE) & (proton_bulk_speed > FILL_VALUE)
    valid_alpha = alpha_density > FILL_VALUE
    valid_thermal = proton_thermal > FILL_VALUE
    
    # Initialize score
    cme_score = 0.0
    
    # Alpha ratio score (0.4 weight)
    if valid_proton and valid_alpha:
        alpha_ratio = alpha_density / proton_density
        cme_score += 0.4 * max(0, min(1, (alpha_ratio - 0.003) / 0.008))
    
    # Speed score (0.4 weight)
    if valid_proton:
        cme_score += 0.4 * max(0, min(1, (proton_bulk_speed - 380) / 200))
    
    # Temperature score (0.1 weight)
    if valid_proton and valid_thermal:
        T_exp = 3.0 * (proton_bulk_speed/100)**0.67 * 1e4
        T_obs = (proton_thermal * 1000)**2 * m_p / (2 * k_B)
        T_ratio = T_obs / T_exp
        if T_ratio < 0.5:
            cme_score += 0.1 * max(0, 1 - 2 * T_ratio)
    
    # Quality score (0.1 weight)
    valid_count = sum([valid_proton, valid_alpha, valid_thermal])
    cme_score += 0.1 * (valid_count / 3.0)
    
    return cme_score


In [5]:
sample_data = prepare_data(files['L1_AUX'])

In [ ]:
sample_data # Aux_data

,epoch_for_cdf,trig_counts,coin_trig_counts,coinc_trig_count_total,angle_tha1,angle_tha2,angle_xgse,angle_ygse,angle_zgse,peak_det_counts,rej_counts,obs_time,spacecraft_xpos,spacecraft_xvel,spacecraft_ypos,spacecraft_yvel,spacecraft_zpos,spacecraft_zvel
0,2025-09-30 00:00:06.106,1,1,1371,0.925418,0.99999,0.085835,89.929637,90.049160,1,0,1.759190e+09,1.661181e+06,-0.050667,-181960.709886,-0.259889,127130.542990,-0.011855
1,2025-09-30 00:00:06.606,2,1,1371,0.925418,0.99999,0.085835,89.929637,90.049160,1,0,1.759190e+09,1.661181e+06,-0.050667,-181960.839830,-0.259889,127130.537062,-0.011855
2,2025-09-30 00:00:07.106,0,0,1371,0.925418,0.99999,0.085835,89.929637,90.049160,0,0,1.759190e+09,1.661181e+06,-0.050667,-181960.969774,-0.259889,127130.531135,-0.011855
3,2025-09-30 00:00:07.606,1,1,1371,0.925418,0.99999,0.085835,89.929637,90.049160,1,0,1.759190e+09,1.661181e+06,-0.050667,-181961.099718,-0.259889,127130.525207,-0.011855
4,2025-09-30 00:00:08.106,1,1,1345,0.925418,0.99999,0.085835,89.929637,90.049160,1,0,1.759190e+09,1.661181e+06,-0.050667,-181961.229663,-0.259889,127130.519279,-0.011855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172782,2025-09-30 23:59:57.854,3,2,496,0.925478,0.99999,0.092885,89.920941,90.048758,1,0,1.759277e+09,1.656636e+06,-0.054460,-204395.726727,-0.259442,126056.235597,-0.013017
172783,2025-09-30 23:59:58.354,0,0,496,0.925478,0.99999,0.092885,89.920941,90.048758,0,0,1.759277e+09,1.656636e+06,-0.054460,-204395.856396,-0.259442,126056.229091,-0.013017
172784,2025-09-30 23:59:58.854,1,1,465,0.925478,0.99999,0.092885,89.920941,90.048758,1,0,1.759277e+09,1.656636e+06,-0.054460,-204395.986117,-0.259442,126056.222582,-0.013017
172785,2025-09-30 23:59:59.354,1,1,465,0.925477,0.99999,0.092885,89.920941,90.048758,1,0,1.759277e+09,1.656636e+06,-0.054460,-204396.115839,-0.259442,126056.216074,-0.013017


In [26]:
# save csv file first 100 rows
sample_data.head(100).to_csv('sample_data.csv', index=False)
L2_BLK_data.head(100).to_csv('L2_BLK_data.csv', index=False)

In [7]:
sample_data.columns

Index(['epoch_for_cdf', 'trig_counts', 'coin_trig_counts',
       'coinc_trig_count_total', 'angle_tha1', 'angle_tha2', 'angle_xgse',
       'angle_ygse', 'angle_zgse', 'peak_det_counts', 'rej_counts', 'obs_time',
       'spacecraft_xpos', 'spacecraft_xvel', 'spacecraft_ypos',
       'spacecraft_yvel', 'spacecraft_zpos', 'spacecraft_zvel'],
      dtype='object')

In [8]:
pycdf.CDF(L1_AUX)

<CDF:
DC_temp_mon: CDF_UINT2 [172787, 3]
ESA1_HV: CDF_UINT2 [172787, 5]
ESA1_HV_set: CDF_UINT2 [172787, 5]
ESA2_HV_mon: CDF_UINT2 [172787, 5]
ESA2_HV_set: CDF_UINT2 [172787, 5]
FEE_cal_chn: CDF_UINT2 [172787]
FEE_cal_mode: CDF_UINT2 [172787]
FEE_volt_mon: CDF_UINT2 [172787, 4]
Focus: CDF_UINT2 [172787, 5]
Focus_set: CDF_UINT2 [172787, 5]
HV_PCB_temp_mon: CDF_UINT2 [172787, 5]
HV_pack_mon: CDF_UINT2 [172787, 2]
HW_LLD: CDF_UINT2 [172787, 5]
MCP1_HV: CDF_UINT2 [172787, 5]
MCP1_HV_set: CDF_UINT2 [172787]
MCP1_PHD_mode: CDF_UINT2 [172787]
MCP1_curr_mon: CDF_UINT2 [172787, 5]
MCP1_curr_override: CDF_UINT2 [172787]
MCP1_curr_trip: CDF_UINT2 [172787]
MCP1_spike_override: CDF_UINT2 [172787]
MCP1_spike_trip: CDF_UINT2 [172787]
MCP1_start_comm: CDF_UINT2 [172787]
MCP1_status: CDF_UINT2 [172787]
MCP1_trip_check: CDF_UINT2 [172787]
MCP2_HV_mon: CDF_UINT2 [172787, 5]
MCP2_HV_set: CDF_UINT2 [172787]
MCP2_PHD_mode: CDF_UINT2 [172787]
MCP2_curr_mon: CDF_UINT2 [172787, 5]
MCP2_curr_override: CDF_UINT2 

In [9]:
pycdf.CDF(L1_TH1)

<CDF:
THA-1_spec: CDF_UINT2 [172787, 5, 4, 470]
epoch_for_cdf: CDF_EPOCH [172787, 5]
fpga_ticks: CDF_DOUBLE [172787]
frame_num: CDF_UINT4 [172787]
obs_time: CDF_DOUBLE [172787, 5]
>

In [10]:
pycdf.CDF(L1_TH2)

<CDF:
THA-2_spec: CDF_UINT2 [172787, 5, 160]
epoch_for_cdf: CDF_EPOCH [172787, 5]
fpga_ticks: CDF_DOUBLE [172787]
frame_num: CDF_UINT4 [172787]
obs_time: CDF_DOUBLE [172787, 5]
>

In [11]:
pycdf.CDF(L2_BLK)

<CDF:
alpha_bulk_speed: CDF_DOUBLE [17275]
alpha_density: CDF_DOUBLE [17275]
alpha_thermal: CDF_DOUBLE [17275]
bulk_a_uncer: CDF_DOUBLE [17275]
bulk_p_uncer: CDF_DOUBLE [17275]
epoch_for_cdf_mod: CDF_EPOCH [17275]
numden_a_uncer: CDF_DOUBLE [17275]
numden_p_uncer: CDF_DOUBLE [17275]
proton_bulk_speed: CDF_DOUBLE [17275]
proton_density: CDF_DOUBLE [17275]
proton_thermal: CDF_DOUBLE [17275]
proton_xvelocity: CDF_DOUBLE [17275]
proton_yvelocity: CDF_DOUBLE [17275]
proton_zvelocity: CDF_DOUBLE [17275]
spacecraft_xpos: CDF_DOUBLE [17275]
spacecraft_ypos: CDF_DOUBLE [17275]
spacecraft_zpos: CDF_DOUBLE [17275]
thermal_a_uncer: CDF_DOUBLE [17275]
thermal_p_uncer: CDF_DOUBLE [17275]
>

In [12]:
pycdf.CDF(L2_TH2)


<CDF:
energy_center_mod: CDF_DOUBLE [17275, 50]
energy_uncer: CDF_DOUBLE [] NRV
epoch_for_cdf_mod: CDF_EPOCH [17275]
flux_uncer: CDF_DOUBLE [] NRV
integrated_flux_mod: CDF_DOUBLE [17275, 50]
integrated_flux_s15_mod: CDF_DOUBLE [17275, 50]
integrated_flux_s16_mod: CDF_DOUBLE [17275, 50]
integrated_flux_s17_mod: CDF_DOUBLE [17275, 50]
integrated_flux_s18_mod: CDF_DOUBLE [17275, 50]
integrated_flux_s19_mod: CDF_DOUBLE [17275, 50]
spacecraft_xpos: CDF_DOUBLE [17275]
spacecraft_ypos: CDF_DOUBLE [17275]
spacecraft_zpos: CDF_DOUBLE [17275]
sun_angle_tha2: CDF_DOUBLE [17275, 32, 3]
>

In [14]:
from src.processing_data import *
L2_BLK_data = prepare_data(files['L2_BLK'])

In [15]:
L2_BLK_data.columns

Index(['time', 'proton_bulk_speed', 'alpha_bulk_speed', 'alpha_density',
       'alpha_thermal', 'proton_density', 'proton_thermal', 'proton_xvelocity',
       'proton_yvelocity', 'proton_zvelocity', 'spacecraft_xpos',
       'spacecraft_ypos', 'spacecraft_zpos'],
      dtype='object')

In [16]:
sample_data.columns

Index(['epoch_for_cdf', 'trig_counts', 'coin_trig_counts',
       'coinc_trig_count_total', 'angle_tha1', 'angle_tha2', 'angle_xgse',
       'angle_ygse', 'angle_zgse', 'peak_det_counts', 'rej_counts', 'obs_time',
       'spacecraft_xpos', 'spacecraft_xvel', 'spacecraft_ypos',
       'spacecraft_yvel', 'spacecraft_zpos', 'spacecraft_zvel'],
      dtype='object')

In [18]:
L2_BLK_data['time'].dtypes, sample_data['epoch_for_cdf'].dtypes

(dtype('<M8[ns]'), dtype('<M8[ns]'))

In [ ]:
# Timestamp('2025-09-30 00:00:28.606000') --> 0-shape
# approx 5sec gap
L2_BLK_data['time'][1]

Timestamp('2025-09-30 00:00:33.606000')

In [ ]:
# 2025-09-30 00:00:06.106000 --> 0-shape
# approx 0.5sec gap
sample_data['epoch_for_cdf'][1]

Timestamp('2025-09-30 00:00:06.606000')

In [30]:
sample_data

,epoch_for_cdf,trig_counts,coin_trig_counts,coinc_trig_count_total,angle_tha1,angle_tha2,angle_xgse,angle_ygse,angle_zgse,peak_det_counts,rej_counts,obs_time,spacecraft_xpos,spacecraft_xvel,spacecraft_ypos,spacecraft_yvel,spacecraft_zpos,spacecraft_zvel
0,2025-09-30 00:00:06.106,1,1,1371,0.925418,0.99999,0.085835,89.929637,90.049160,1,0,1.759190e+09,1.661181e+06,-0.050667,-181960.709886,-0.259889,127130.542990,-0.011855
1,2025-09-30 00:00:06.606,2,1,1371,0.925418,0.99999,0.085835,89.929637,90.049160,1,0,1.759190e+09,1.661181e+06,-0.050667,-181960.839830,-0.259889,127130.537062,-0.011855
2,2025-09-30 00:00:07.106,0,0,1371,0.925418,0.99999,0.085835,89.929637,90.049160,0,0,1.759190e+09,1.661181e+06,-0.050667,-181960.969774,-0.259889,127130.531135,-0.011855
3,2025-09-30 00:00:07.606,1,1,1371,0.925418,0.99999,0.085835,89.929637,90.049160,1,0,1.759190e+09,1.661181e+06,-0.050667,-181961.099718,-0.259889,127130.525207,-0.011855
4,2025-09-30 00:00:08.106,1,1,1345,0.925418,0.99999,0.085835,89.929637,90.049160,1,0,1.759190e+09,1.661181e+06,-0.050667,-181961.229663,-0.259889,127130.519279,-0.011855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172782,2025-09-30 23:59:57.854,3,2,496,0.925478,0.99999,0.092885,89.920941,90.048758,1,0,1.759277e+09,1.656636e+06,-0.054460,-204395.726727,-0.259442,126056.235597,-0.013017
172783,2025-09-30 23:59:58.354,0,0,496,0.925478,0.99999,0.092885,89.920941,90.048758,0,0,1.759277e+09,1.656636e+06,-0.054460,-204395.856396,-0.259442,126056.229091,-0.013017
172784,2025-09-30 23:59:58.854,1,1,465,0.925478,0.99999,0.092885,89.920941,90.048758,1,0,1.759277e+09,1.656636e+06,-0.054460,-204395.986117,-0.259442,126056.222582,-0.013017
172785,2025-09-30 23:59:59.354,1,1,465,0.925477,0.99999,0.092885,89.920941,90.048758,1,0,1.759277e+09,1.656636e+06,-0.054460,-204396.115839,-0.259442,126056.216074,-0.013017


In [53]:
L2_BLK_data

,time,proton_bulk_speed,alpha_bulk_speed,alpha_density,alpha_thermal,proton_density,proton_thermal,proton_xvelocity,proton_yvelocity,proton_zvelocity,spacecraft_xpos,spacecraft_ypos,spacecraft_zpos
0,2025-09-30 00:00:28.606,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-11.191019,17.306730,1.661180e+06,-181966.557429,127130.276242
1,2025-09-30 00:00:33.606,4.011090e+02,4.112124e+02,2.586606e-02,7.391349e+01,6.956146e+00,8.364392e+01,-4.011090e+02,-1.997462,10.329966,1.661179e+06,-181967.856897,127130.216963
2,2025-09-30 00:00:38.606,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-3.246582,16.831908,1.661179e+06,-181969.156313,127130.157687
3,2025-09-30 00:00:43.606,4.001531e+02,3.461118e+02,7.361456e-02,1.058376e+02,6.566578e+00,8.083200e+01,-4.001531e+02,-1.438449,18.214916,1.661179e+06,-181970.455781,127130.098407
4,2025-09-30 00:00:48.606,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-10.052654,15.604581,1.661179e+06,-181971.755223,127130.039128
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17270,2025-09-30 23:59:39.354,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,36.829296,30.846123,1.656637e+06,-204390.926964,126056.476410
17271,2025-09-30 23:59:44.354,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,38.290054,50.262459,1.656637e+06,-204392.224176,126056.411327
17272,2025-09-30 23:59:49.354,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,36.716742,52.877282,1.656637e+06,-204393.521415,126056.346242
17273,2025-09-30 23:59:54.354,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,28.219071,56.549723,1.656636e+06,-204394.818627,126056.281158


In [38]:
def cme_velocity(L2_BLK):
    return np.sqrt(L2_BLK['proton_xvelocity']**2 + L2_BLK['proton_yvelocity']**2 + L2_BLK['proton_zvelocity']**2)
val = cme_velocity(L2_BLK_data)


In [52]:
L2_BLK_data['proton_xvelocity'].isnull().sum(), L2_BLK_data['proton_yvelocity'].isnull().sum(), L2_BLK_data['proton_zvelocity'].isnull().sum()

(0, 0, 0)

In [39]:
val.min()

365.3751503854251

In [40]:
val.max()

1e+31

In [49]:
val.isnull().sum()

0

In [47]:
val.unique()

array([1.00000000e+31, 4.01246929e+02, 4.00570054e+02, ...,
       7.60901586e+02, 7.55096318e+02, 7.22355943e+02])

In [42]:
val.describe()

count    1.727500e+04
mean     6.919247e+30
std      4.617112e+30
min      3.653752e+02
25%      6.495241e+02
50%      1.000000e+31
75%      1.000000e+31
max      1.000000e+31
dtype: float64

In [43]:
L2_BLK_data


,time,proton_bulk_speed,alpha_bulk_speed,alpha_density,alpha_thermal,proton_density,proton_thermal,proton_xvelocity,proton_yvelocity,proton_zvelocity,spacecraft_xpos,spacecraft_ypos,spacecraft_zpos
0,2025-09-30 00:00:28.606,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-11.191019,17.306730,1.661180e+06,-181966.557429,127130.276242
1,2025-09-30 00:00:33.606,4.011090e+02,4.112124e+02,2.586606e-02,7.391349e+01,6.956146e+00,8.364392e+01,-4.011090e+02,-1.997462,10.329966,1.661179e+06,-181967.856897,127130.216963
2,2025-09-30 00:00:38.606,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-3.246582,16.831908,1.661179e+06,-181969.156313,127130.157687
3,2025-09-30 00:00:43.606,4.001531e+02,3.461118e+02,7.361456e-02,1.058376e+02,6.566578e+00,8.083200e+01,-4.001531e+02,-1.438449,18.214916,1.661179e+06,-181970.455781,127130.098407
4,2025-09-30 00:00:48.606,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-10.052654,15.604581,1.661179e+06,-181971.755223,127130.039128
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17270,2025-09-30 23:59:39.354,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,36.829296,30.846123,1.656637e+06,-204390.926964,126056.476410
17271,2025-09-30 23:59:44.354,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,38.290054,50.262459,1.656637e+06,-204392.224176,126056.411327
17272,2025-09-30 23:59:49.354,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,36.716742,52.877282,1.656637e+06,-204393.521415,126056.346242
17273,2025-09-30 23:59:54.354,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,28.219071,56.549723,1.656636e+06,-204394.818627,126056.281158


In [20]:
import numpy as np
import pandas as pd

def calculate_cme_score(proton_density, proton_bulk_speed, proton_thermal, alpha_density):
    """
    Vectorized CME score calculator that accepts pandas Series or numpy arrays.
    Returns a pandas Series of scores (same index as input if inputs are Series).
    """
    # convert to numpy arrays (keep index if pandas supplied)
    is_series = isinstance(proton_density, (pd.Series, pd.Index))
    idx = proton_density.index if is_series else None

    pd_arr = np.asarray(proton_density, dtype=float)
    v_arr = np.asarray(proton_bulk_speed, dtype=float)
    t_arr = np.asarray(proton_thermal, dtype=float)
    a_arr = np.asarray(alpha_density, dtype=float)

    FILL_VALUE = -1.0e31
    k_B, m_p = 1.381e-23, 1.673e-27

    # Validity masks
    valid_proton = (pd_arr > FILL_VALUE) & (v_arr > FILL_VALUE)
    valid_alpha  = (a_arr  > FILL_VALUE) 
    valid_thermal= (t_arr  > FILL_VALUE)

    # Start with zeros
    score = np.zeros_like(pd_arr, dtype=float)

    # --- Alpha ratio score (0.4 weight) ---
    # Avoid divide-by-zero: require proton_density > 0
    denom_ok = pd_arr > 0
    mask_alpha = valid_proton & valid_alpha & denom_ok
    alpha_ratio = np.zeros_like(pd_arr, dtype=float)
    alpha_ratio[mask_alpha] = a_arr[mask_alpha] / pd_arr[mask_alpha]
    # scaled into [0,1] where 0.003->0 and 0.011->1 (as in your original)
    alpha_scaled = np.clip((alpha_ratio - 0.003) / 0.008, 0, 1)
    score += 0.4 * alpha_scaled

    # --- Speed score (0.4 weight) ---
    mask_speed = valid_proton
    speed_scaled = np.zeros_like(pd_arr, dtype=float)
    speed_scaled[mask_speed] = np.clip((v_arr[mask_speed] - 380) / 200, 0, 1)
    score += 0.4 * speed_scaled

    # --- Temperature score (0.1 weight) ---
    # Use vector formulas; compute only where valid_proton & valid_thermal
    mask_temp = valid_proton & valid_thermal
    if np.any(mask_temp):
        # Expected temperature (T_exp) (matches your formula)
        T_exp = 3.0 * (v_arr[mask_temp] / 100.0) ** 0.67 * 1e4  # same units as original
        # Observed temperature (T_obs) using given relation
        # (proton_thermal * 1000)**2 * m_p / (2 * k_B)
        T_obs = ((t_arr[mask_temp] * 1000.0) ** 2) * m_p / (2.0 * k_B)
        # Avoid negative/zero T_exp
        safe_T_exp = np.where(T_exp > 0, T_exp, np.nan)
        T_ratio = T_obs / safe_T_exp
        # condition from original: if T_ratio < 0.5 then add 0.1 * (1 - 2*T_ratio) clipped to [0,0.1]
        temp_contrib = np.zeros_like(T_ratio)
        cond = T_ratio < 0.5
        temp_contrib[cond] = np.maximum(0.0, 1.0 - 2.0 * T_ratio[cond])  # in [0,1]
        score[mask_temp] += 0.1 * temp_contrib

    # --- Quality score (0.1 weight) ---
    valid_count = valid_proton.astype(int) + valid_alpha.astype(int) + valid_thermal.astype(int)
    score += 0.1 * (valid_count / 3.0)

    # Return pandas Series if input was Series, else numpy array
    if idx is not None:
        return pd.Series(score, index=idx)
    return score

# Example usage with a DataFrame 'df':
df['cme_score'] = calculate_cme_score(df['proton_density'], df['proton_bulk_speed'], df['proton_thermal'], df['alpha_density'])


In [22]:
df['cme_score'].unique()

array([0.        , 0.17814027, 0.54030623, 0.27232018, 0.32002558,
       0.17531174, 0.2707143 , 0.20380636, 0.30026642, 0.20783225,
       0.28035549, 0.33387083, 0.35482982, 0.14364103, 0.28824467,
       0.14995149, 0.33257492, 0.14123775, 0.24842973, 0.14357076,
       0.18384006, 0.14170907, 0.24003314, 0.14253586, 0.16144135,
       0.2579761 , 0.14248519, 0.14491464, 0.16249582, 0.14758929,
       0.1394516 , 0.16855124, 0.13878477, 0.15808048, 0.137212  ,
       0.18787772, 0.13673513, 0.15438034, 0.14234538, 0.24914441,
       0.2411059 , 0.13893935, 0.13551567, 0.13563623, 0.21641235,
       0.13218218, 0.13300851, 0.26240694, 0.21721643, 0.15619771,
       0.15035386, 0.31131703, 0.21750763, 0.29573461, 0.27720791,
       0.28416895, 0.15927784, 0.17053181, 0.22398989, 0.23742043,
       0.1351356 , 0.20606328, 0.16178902, 0.29464028, 0.21677579,
       0.25484603, 0.25847384, 0.1386442 ])

In [35]:
def detect_cme_speed_signatures(proton_speeds):
    # Enhancement scores (same length)
    scores = np.maximum(0, np.minimum(1, (proton_speeds - 380) / 200))

    # Speed changes
    speed_changes = np.diff(proton_speeds, prepend=proton_speeds[0])
    rapid_increases = speed_changes > 5

    # Acceleration
    acceleration_changes = np.diff(speed_changes, prepend=speed_changes[0])
    accelerations = acceleration_changes > 2

    return scores, rapid_increases, accelerations, acceleration_changes

In [24]:
df.columns

Index(['epoch_for_cdf_mod', 'proton_density', 'numden_p_uncer',
       'proton_bulk_speed', 'bulk_p_uncer', 'proton_xvelocity',
       'proton_yvelocity', 'proton_zvelocity', 'proton_thermal',
       'thermal_p_uncer', 'alpha_density', 'numden_a_uncer',
       'alpha_bulk_speed', 'bulk_a_uncer', 'alpha_thermal', 'thermal_a_uncer',
       'spacecraft_xpos', 'spacecraft_ypos', 'spacecraft_zpos', 'cme_score'],
      dtype='object')

In [36]:
df['score'], df['rapid_increases'], df['accelerations'], df['acceleration_changes'] = detect_cme_speed_signatures(df['proton_bulk_speed'])

In [41]:
df.columns

Index(['epoch_for_cdf_mod', 'proton_density', 'numden_p_uncer',
       'proton_bulk_speed', 'bulk_p_uncer', 'proton_xvelocity',
       'proton_yvelocity', 'proton_zvelocity', 'proton_thermal',
       'thermal_p_uncer', 'alpha_density', 'numden_a_uncer',
       'alpha_bulk_speed', 'bulk_a_uncer', 'alpha_thermal', 'thermal_a_uncer',
       'spacecraft_xpos', 'spacecraft_ypos', 'spacecraft_zpos', 'cme_score',
       'score', 'rapid_increases', 'accelerations', 'acceleration_changes'],
      dtype='object')

In [38]:
# df['score'], df['rapid_increases'], 
df['acceleration_changes']

-1.000000e+31    28
 2.000000e+31    18
 1.000000e+31    11
-2.000000e+31     9
 0.000000e+00     5
-3.168505e+00     1
 1.772117e+00     1
-2.299952e+00     1
 4.118351e+00     1
-2.646806e+00     1
 3.418176e+00     1
 2.715269e+00     1
 1.153109e+00     1
 4.428252e+00     1
 2.994310e-01     1
-1.995394e+00     1
-3.015763e+00     1
 4.099927e-01     1
-3.610926e+00     1
-3.176068e+00     1
 6.905570e+00     1
-1.006400e+00     1
 2.065243e+00     1
-3.339685e+00     1
 2.796042e+00     1
-1.423076e+00     1
 1.302971e+00     1
-3.123983e-01     1
 1.437337e-01     1
-5.778012e-01     1
-1.462648e+00     1
 9.953481e-01     1
-9.583051e-01     1
 5.175816e+00     1
Name: acceleration_changes, dtype: int64

In [39]:
def detect_cme_speed_signatures(proton_speeds):
    # Enhancement scores
    scores = np.maximum(0, np.minimum(1, (proton_speeds - 380) / 200))
    
    # Speed changes
    if len(proton_speeds) > 1:
        speed_changes = np.diff(proton_speeds)
        rapid_increases = speed_changes > 5  # 5 km/s threshold
        accelerations = np.diff(speed_changes) > 2  # acceleration
    
    return scores, rapid_increases, accelerations

In [40]:
df['cme_speed'] = detect_cme_speed_signatures(df['proton_bulk_speed'])

ValueError: Length of values (3) does not match length of index (100)

In [16]:
import pandas as pd 
df = pd.read_csv('L2_BLK.csv')
df.columns

Index(['epoch_for_cdf_mod', 'proton_density', 'numden_p_uncer',
       'proton_bulk_speed', 'bulk_p_uncer', 'proton_xvelocity',
       'proton_yvelocity', 'proton_zvelocity', 'proton_thermal',
       'thermal_p_uncer', 'alpha_density', 'numden_a_uncer',
       'alpha_bulk_speed', 'bulk_a_uncer', 'alpha_thermal', 'thermal_a_uncer',
       'spacecraft_xpos', 'spacecraft_ypos', 'spacecraft_zpos'],
      dtype='object')

In [ ]:
def detect_cme_thresholds(alpha_density, proton_density, proton_bulk_speed, proton_thermal):
    """
    Detect CME ejection based on threshold values.

    Parameters
    ----------
    alpha_density : array-like
        Alpha particle density (particles/cm³)
    proton_density : array-like
        Proton density (particles/cm³)
    proton_bulk_speed : array-like
        Proton bulk speed (km/s)
    proton_thermal : array-like
        Proton thermal speed (km/s)

    Returns
    -------
    cme_flags : dict of bool arrays
        Dictionary with boolean arrays indicating if thresholds are exceeded.
        Keys: 'alpha_proton_ratio', 'bulk_speed', 'temperature_depression'
    """
    # Physical constants
    m_p = 1.673e-27  # kg
    k_B = 1.381e-23  # J/K

    # Alpha to proton ratio
    alpha_proton_ratio = np.divide(alpha_density, proton_density, 
                                  out=np.zeros_like(alpha_density), where=proton_density!=0)
    alpha_flag = alpha_proton_ratio > 0.08  # Threshold

    # Bulk speed threshold
    speed_flag = proton_bulk_speed > 450  # km/s threshold

    # Temperature depression threshold
    T_exp = 3.0 * (proton_bulk_speed / 100.0)**0.67 * 1e4  # Expected temperature (K)
    T_obs = ((proton_thermal * 1000) ** 2) * m_p / (2 * k_B)  # Observed temperature (K)
    temp_flag = T_obs < 0.5 * T_exp

    return alpha_flag, speed_flag, temp_flag


# Example usage:
# alpha_density = np.array([...])
# proton_density = np.array([...])
# proton_bulk_speed = np.array([...])
# proton_thermal = np.array([...])

df[] = detect_cme_thresholds(df['alpha_density'], df['proton_density'], df['proton_bulk_speed'], df['proton_thermal'])



In [ ]:
value

{'alpha_proton_ratio': 0      True
 1     False
 2      True
 3     False
 4      True
       ...  
 95     True
 96     True
 97    False
 98     True
 99    False
 Length: 100, dtype: bool,
 'bulk_speed': 0     False
 1     False
 2     False
 3     False
 4     False
       ...  
 95    False
 96    False
 97    False
 98    False
 99    False
 Name: proton_bulk_speed, Length: 100, dtype: bool,
 'temperature_depression': 0     False
 1     False
 2     False
 3     False
 4     False
       ...  
 95    False
 96    False
 97    False
 98    False
 99    False
 Length: 100, dtype: bool}

In [43]:
df.columns

Index(['epoch_for_cdf_mod', 'proton_density', 'numden_p_uncer',
       'proton_bulk_speed', 'bulk_p_uncer', 'proton_xvelocity',
       'proton_yvelocity', 'proton_zvelocity', 'proton_thermal',
       'thermal_p_uncer', 'alpha_density', 'numden_a_uncer',
       'alpha_bulk_speed', 'bulk_a_uncer', 'alpha_thermal', 'thermal_a_uncer',
       'spacecraft_xpos', 'spacecraft_ypos', 'spacecraft_zpos', 'cme_score',
       'score', 'rapid_increases', 'accelerations', 'acceleration_changes'],
      dtype='object')

In [ ]:
import numpy as np

def compute_empirical_threshold(data, method='percentile', param=95, nan_fill_value=-1e31):
    """
    Compute an empirical threshold for CME detection from a 1D data array.

    Parameters
    ----------
    data : array-like
        Input measurement array (e.g. alpha/proton ratio, proton speed).
    method : str, optional
        'percentile' or 'mean_sigma'. Default is 'percentile'.
    param : float, optional
        If method=='percentile', the percentile (0–100) to use.
        If method=='mean_sigma', the number of standard deviations (sigma).
    nan_fill_value : float, optional
        Value in data that indicates missing or fill; these are ignored.

    Returns
    -------
    threshold : float
        Computed threshold value.
    """
    arr = np.array(data, dtype=float)
    # Mask out fill values and NaNs
    valid = (~np.isnan(arr)) & (arr != nan_fill_value)
    clean = arr[valid]
    if clean.size == 0:
        raise ValueError("No valid data points")

    if method == 'percentile':
        # e.g. 95th percentile
        return np.percentile(clean, param)
    elif method == 'mean_sigma':
        # mean + param * std
        mu = clean.mean()
        sigma = clean.std()
        return mu + param * sigma
    else:
        raise ValueError("method must be 'percentile' or 'mean_sigma'")

# Example usage for your ASPEX-SWIS data:
# compute_empirical_threshold('L2_BLK.csv')
# import pandas as pd

# # Load L2_BLK.csv
    df = pd.read_csv('L2_BLK.csv')

    # 1. Alpha/proton ratio threshold at 90th percentile
    alpha_pr = df['alpha_density'] / df['proton_density']
    alpha_thr = compute_empirical_threshold(alpha_pr, method='percentile', param=90)
    print(f"Alpha/Proton ratio threshold (90th pct): {alpha_thr:.4f}")

    # 2. Proton speed threshold as mean + 2σ
    speed_thr = compute_empirical_threshold(df['proton_bulk_speed'], method='mean_sigma', param=2)
    print(f"Proton speed threshold (mean+2σ): {speed_thr:.1f} km/s")

    # 3. Temperature depression ratio threshold at 95th percentile
    # First compute temperature ratio array
    m_p = 1.673e-27
    k_B = 1.381e-23
    T_exp = 3.0 * (df['proton_bulk_speed'] / 100)**0.67 * 1e4
    T_obs = (df['proton_thermal'] * 1000)**2 * m_p / (2 * k_B)
    T_ratio = T_obs / T_exp
    temp_thr = compute_empirical_threshold(T_ratio, method='percentile', param=95)
    print(f"Temperature ratio threshold (95th pct): {temp_thr:.2f}")


Alpha/Proton ratio threshold (90th pct): 1.0000
Proton speed threshold (mean+2σ): 403.1 km/s
Temperature ratio threshold (95th pct): 6.04


In [ ]:
df = pd.read_csv('L2_BLK.csv')

,epoch_for_cdf_mod,proton_density,numden_p_uncer,proton_bulk_speed,bulk_p_uncer,proton_xvelocity,proton_yvelocity,proton_zvelocity,proton_thermal,thermal_p_uncer,alpha_density,numden_a_uncer,alpha_bulk_speed,bulk_a_uncer,alpha_thermal,thermal_a_uncer,spacecraft_xpos,spacecraft_ypos,spacecraft_zpos
0,2025-09-30 00:00:28.606,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-11.191019,17.306730,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,1.661180e+06,-181966.557429,127130.276242
1,2025-09-30 00:00:33.606,6.956146e+00,-1.000000e+31,4.011090e+02,-1.000000e+31,-4.011090e+02,-1.997462,10.329966,8.364392e+01,-1.000000e+31,2.586606e-02,-1.000000e+31,4.112124e+02,-1.000000e+31,7.391349e+01,-1.000000e+31,1.661179e+06,-181967.856897,127130.216963
2,2025-09-30 00:00:38.606,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-3.246582,16.831908,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,1.661179e+06,-181969.156313,127130.157687
3,2025-09-30 00:00:43.606,6.566578e+00,-1.000000e+31,4.001531e+02,-1.000000e+31,-4.001531e+02,-1.438449,18.214916,8.083200e+01,-1.000000e+31,7.361456e-02,-1.000000e+31,3.461118e+02,-1.000000e+31,1.058376e+02,-1.000000e+31,1.661179e+06,-181970.455781,127130.098407
4,2025-09-30 00:00:48.606,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-10.052654,15.604581,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,1.661179e+06,-181971.755223,127130.039128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2025-09-30 00:08:23.610,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,2.920279,12.974129,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,1.661156e+06,-182090.005058,127124.643323
96,2025-09-30 00:08:28.610,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,4.107720,12.117373,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,1.661155e+06,-182091.304544,127124.584011
97,2025-09-30 00:08:33.610,7.579596e+00,-1.000000e+31,3.993536e+02,-1.000000e+31,-3.993536e+02,1.491662,5.417377,8.400806e+01,-1.000000e+31,4.089443e-02,-1.000000e+31,3.623847e+02,-1.000000e+31,1.122482e+02,-1.000000e+31,1.661155e+06,-182092.603978,127124.524701
98,2025-09-30 00:08:38.610,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,0.786263,16.478029,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,1.661155e+06,-182093.903411,127124.465391


In [58]:
import pandas as pd
df = pd.read_csv('final_all_values.csv')
df

C:\Users\piyus\AppData\Local\Temp\ipykernel_7884\1667219309.py:2: DtypeWarning: Columns (425) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('final_all_values.csv')


,obs_time_0_L1,obs_time_1_L1,obs_time_2_L1,obs_time_3_L1,obs_time_4_L1,epoch_for_cdf_0_L1,epoch_for_cdf_1_L1,epoch_for_cdf_2_L1,epoch_for_cdf_3_L1,epoch_for_cdf_4_L1,...,alpha_density_L2,numden_a_uncer_L2,alpha_bulk_speed_L2,bulk_a_uncer_L2,alpha_thermal_L2,thermal_a_uncer_L2,spacecraft_xpos_L2,spacecraft_ypos_L2,spacecraft_zpos_L2,cme_score
0,1.759190e+09,1.759190e+09,1.759190e+09,1.759190e+09,1.759190e+09,2025-09-30 00:00:06.106,2025-09-30 00:00:06.206,2025-09-30 00:00:06.306,2025-09-30 00:00:06.406,2025-09-30 00:00:06.506,...,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,1.661180e+06,-181966.557429,127130.276242,0.000000
1,1.759190e+09,1.759190e+09,1.759190e+09,1.759190e+09,1.759190e+09,2025-09-30 00:00:06.606,2025-09-30 00:00:06.706,2025-09-30 00:00:06.806,2025-09-30 00:00:06.906,2025-09-30 00:00:07.006,...,2.586606e-02,-1.000000e+31,4.112124e+02,-1.000000e+31,7.391349e+01,-1.000000e+31,1.661179e+06,-181967.856897,127130.216963,0.033333
2,1.759190e+09,1.759190e+09,1.759190e+09,1.759190e+09,1.759190e+09,2025-09-30 00:00:07.106,2025-09-30 00:00:07.206,2025-09-30 00:00:07.306,2025-09-30 00:00:07.406,2025-09-30 00:00:07.506,...,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,1.661179e+06,-181969.156313,127130.157687,0.000000
3,1.759190e+09,1.759190e+09,1.759190e+09,1.759190e+09,1.759190e+09,2025-09-30 00:00:07.606,2025-09-30 00:00:07.706,2025-09-30 00:00:07.806,2025-09-30 00:00:07.906,2025-09-30 00:00:08.006,...,7.361456e-02,-1.000000e+31,3.461118e+02,-1.000000e+31,1.058376e+02,-1.000000e+31,1.661179e+06,-181970.455781,127130.098407,0.033333
4,1.759190e+09,1.759190e+09,1.759190e+09,1.759190e+09,1.759190e+09,2025-09-30 00:00:08.106,2025-09-30 00:00:08.206,2025-09-30 00:00:08.306,2025-09-30 00:00:08.406,2025-09-30 00:00:08.506,...,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,-1.000000e+31,1.661179e+06,-181971.755223,127130.039128,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172782,1.759277e+09,1.759277e+09,1.759277e+09,1.759277e+09,1.759277e+09,2025-09-30 23:59:57.854,2025-09-30 23:59:57.954,2025-09-30 23:59:58.054,2025-09-30 23:59:58.154,2025-09-30 23:59:58.254,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
172783,1.759277e+09,1.759277e+09,1.759277e+09,1.759277e+09,1.759277e+09,2025-09-30 23:59:58.354,2025-09-30 23:59:58.454,2025-09-30 23:59:58.554,2025-09-30 23:59:58.654,2025-09-30 23:59:58.754,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
172784,1.759277e+09,1.759277e+09,1.759277e+09,1.759277e+09,1.759277e+09,2025-09-30 23:59:58.854,2025-09-30 23:59:58.954,2025-09-30 23:59:59.054,2025-09-30 23:59:59.154,2025-09-30 23:59:59.254,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
172785,1.759277e+09,1.759277e+09,1.759277e+09,1.759277e+09,1.759277e+09,2025-09-30 23:59:59.354,2025-09-30 23:59:59.454,2025-09-30 23:59:59.554,2025-09-30 23:59:59.654,2025-09-30 23:59:59.754,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000


In [59]:
df.columns

Index(['obs_time_0_L1', 'obs_time_1_L1', 'obs_time_2_L1', 'obs_time_3_L1',
       'obs_time_4_L1', 'epoch_for_cdf_0_L1', 'epoch_for_cdf_1_L1',
       'epoch_for_cdf_2_L1', 'epoch_for_cdf_3_L1', 'epoch_for_cdf_4_L1',
       ...
       'alpha_density_L2', 'numden_a_uncer_L2', 'alpha_bulk_speed_L2',
       'bulk_a_uncer_L2', 'alpha_thermal_L2', 'thermal_a_uncer_L2',
       'spacecraft_xpos_L2', 'spacecraft_ypos_L2', 'spacecraft_zpos_L2',
       'cme_score'],
      dtype='object', length=445)

In [1]:
import pandas as pd
df = pd.read_csv('final_output_01.csv')

In [2]:
df.columns

Index(['scores_data', 'speed_enhancement', 'rapid_increase',
       'acceleration_flag', 'alpha_flag', 'speed_flag', 'temp_flag'],
      dtype='object')

In [3]:
df['scores_data'].max()

0.9

In [ ]:
speed = df['speed_enhancement'].max()

1.0

In [2]:
import pandas as pd
value = pd.read_csv('final_output_01.csv')
value.head(100).to_csv('sample.csv')